In [ ]:
# WARNING: advised to install a specific version, e.g. ampform==0.1.2
%pip install -q ampform[doc,viz] IPython

In [ ]:
%config InlineBackend.figure_formats = ['svg']
import os

from IPython.display import display  # noqa: F401

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

```{autolink-concat}
```

# Analytic continuation

:::{note}

Improvements to analytic continuation in AmpForm are currently being developed in {doc}`compwa-org:report/003` and {doc}`compwa-org:report/004`.

:::

Analytic continuation allows one to handle resonances just below threshold ($m_0 < m_a + m_b$  in Eq. {eq}`relativistic_breit_wigner_with_ff`). In practice, this entails using a specific function for $\rho$ in Eq. {eq}`EnergyDependentWidth`.

## Definitions

Three usual choices for $\rho$ are the following:

In [ ]:
import warnings

import sympy as sp
from IPython.display import Math

warnings.filterwarnings("ignore")

### 1) Break-up momentum

The {func}`~sympy.functions.elementary.miscellaneous.sqrt` or {class}`.ComplexSqrt` of {class}`.BreakupMomentumSquared`:

In [ ]:
from ampform.dynamics import BreakupMomentumSquared

s, m_a, m_b = sp.symbols("s, m_a, m_b", nonnegative=True)
q_squared = BreakupMomentumSquared(s, m_a, m_b)
Math(sp.multiline_latex(q_squared, q_squared.doit(), environment="eqnarray"))

### 2) 'Normal' phase space factor

The 'normal' {class}`.PhaseSpaceFactor` (the denominator makes the difference to {eq}`EnergyDependentWidth`!):

In [ ]:
from ampform.dynamics import PhaseSpaceFactor

rho = PhaseSpaceFactor(s, m_a, m_b)
Math(sp.multiline_latex(rho, rho.evaluate(), environment="eqnarray"))

### 3) 'Complex' phase space factor

A {class}`.PhaseSpaceFactorComplex` that uses {class}`.ComplexSqrt`:

In [ ]:
from ampform.dynamics import PhaseSpaceFactorComplex

rho_c = PhaseSpaceFactorComplex(s, m_a, m_b)
Math(sp.multiline_latex(rho_c, rho_c.evaluate(), environment="eqnarray"))

### 4) 'Analytic continuation' of the phase space factor

The following 'case-by-case' **analytic continuation** for decay products with an _equal_ mass, {class}`.EqualMassPhaseSpaceFactor`:

In [ ]:
from ampform.dynamics import EqualMassPhaseSpaceFactor

rho_ac = EqualMassPhaseSpaceFactor(s, m_a, m_b)
Math(sp.multiline_latex(rho_ac, rho_ac.evaluate(), environment="eqnarray"))

with

In [ ]:
from ampform.dynamics import PhaseSpaceFactorAbs

rho_hat = PhaseSpaceFactorAbs(s, m_a, m_b)
Math(sp.multiline_latex(rho_hat, rho_hat.evaluate(), environment="eqnarray"))

(Mind the absolute value.)

### 5) Chew-Mandelstam for $S$-waves

A {class}`.PhaseSpaceFactorSWave` that uses {func}`.chew_mandelstam_s_wave`:

In [ ]:
from ampform.dynamics import PhaseSpaceFactorSWave

rho_cm = PhaseSpaceFactorSWave(s, m_a, m_b)
Math(sp.multiline_latex(rho_cm, rho_cm.evaluate(), environment="eqnarray"))

## Visualization

```{autolink-skip}
```

In [ ]:
%matplotlib widget

In [ ]:
import matplotlib.pyplot as plt
import mpl_interactions.ipyplot as iplt
import numpy as np

import symplot

In [ ]:
from ampform.sympy.math import ComplexSqrt

m = sp.Symbol("m", nonnegative=True)
rho_c = PhaseSpaceFactorComplex(m**2, m_a, m_b)
rho_cm = PhaseSpaceFactorSWave(m**2, m_a, m_b)
rho_ac = EqualMassPhaseSpaceFactor(m**2, m_a, m_b)
np_rho_c, sliders = symplot.prepare_sliders(
    plot_symbol=m, expression=rho_c.doit()
)
np_rho_ac = sp.lambdify((m, m_a, m_b), rho_ac.doit())
np_rho_cm = sp.lambdify((m, m_a, m_b), rho_cm.doit())
np_breakup_momentum = sp.lambdify(
    (m, m_a, m_b),
    ComplexSqrt(q_squared.subs(s, m**2).doit()) / (8 * sp.pi),
)

{{ run_interactive }}

In [ ]:
plot_domain = np.linspace(0, 3, 500)
sliders.set_ranges(
    m_a=(0, 2, 200),
    m_b=(0, 2, 200),
)
sliders.set_values(
    m_a=0.3,
    m_b=0.75,
)

In [ ]:
fig, axes = plt.subplots(
    ncols=2,
    nrows=2,
    figsize=[8, 8],
    sharex=True,
    sharey=True,
)
fig.canvas.toolbar_visible = False

(ax_q, ax_rho), (ax_rho_ac, ax_rho_cm) = axes
for ax in [ax_q, ax_rho, ax_rho_cm, ax_rho_ac]:
    ax.set_xlabel("$m$")
    ax.set_yticks([])
for ax in [ax_rho_cm, ax_rho_ac]:
    ax.set_yticks([])

ylim = (-0.002, 0.03)


def func_imag(func, *args, **kwargs):
    return lambda *args, **kwargs: func(*args, **kwargs).imag


def func_real(func, *args, **kwargs):
    return lambda *args, **kwargs: func(*args, **kwargs).real


q_math = ComplexSqrt(sp.Symbol("q^2")) / (8 * sp.pi)
ax_q.set_title(f"${sp.latex(q_math)}$")
controls = iplt.plot(
    plot_domain,
    func_real(np_breakup_momentum),
    label="real",
    **sliders,
    ylim=ylim,
    ax=ax_q,
    alpha=0.7,
)
iplt.plot(
    plot_domain,
    func_imag(np_breakup_momentum),
    label="imaginary",
    controls=controls,
    ylim=ylim,
    ax=ax_q,
    alpha=0.7,
)

ax_rho.set_title(f"${sp.latex(rho_c)}$")
iplt.plot(
    plot_domain,
    func_real(np_rho_c),
    label="real",
    controls=controls,
    ylim=ylim,
    ax=ax_rho,
    alpha=0.7,
)
iplt.plot(
    plot_domain,
    func_imag(np_rho_c),
    label="imaginary",
    controls=controls,
    ylim=ylim,
    ax=ax_rho,
    alpha=0.7,
)

ax_rho_ac.set_title(R"equal mass $\rho^\mathrm{eq}(m^2)$")
iplt.plot(
    plot_domain,
    func_real(np_rho_ac),
    label="real",
    controls=controls,
    ylim=ylim,
    ax=ax_rho_ac,
    alpha=0.7,
)
iplt.plot(
    plot_domain,
    func_imag(np_rho_ac),
    label="imaginary",
    controls=controls,
    ylim=ylim,
    ax=ax_rho_ac,
    alpha=0.7,
)

ax_rho_cm.set_title(R"Chew-Mandelstam $\rho^\mathrm{CM}(m^2)$")
iplt.plot(
    plot_domain,
    func_real(np_rho_cm),
    label="real",
    controls=controls,
    ylim=ylim,
    ax=ax_rho_cm,
    alpha=0.7,
)
iplt.plot(
    plot_domain,
    func_imag(np_rho_cm),
    label="imaginary",
    controls=controls,
    ylim=ylim,
    ax=ax_rho_cm,
    alpha=0.7,
)

fig.tight_layout()
plt.legend(loc="upper right")
plt.show()

In [ ]:
if STATIC_WEB_PAGE:
    from IPython.display import SVG

    output_file = "analytic-continuation.svg"
    plt.savefig(output_file)
    display(SVG(output_file))